<H1>MTA Passenger Data Analisys</H1>

Evaluate turnstile data to look for optimal locations for positioning staff for fundraising

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as mat
import datetime as dt

In [2]:
# Read the datafiles
data_path = '../data/'
data_file_list = ['turnstile_191102.txt',
                 'turnstile_191109.txt',
                 'turnstile_191116.txt',
                 'turnstile_191130.txt',
                 'turnstile_191207.txt',
                 'turnstile_191214.txt',
                 'turnstile_191221.txt',
                 'turnstile_191228.txt'
                 'turnstile_200104.txt']

booth_key_filename = 'Remote-Booth-Station.csv'


In [4]:
turnstiles1 = pd.read_csv(data_path + data_file_list[4])
booth_key = pd.read_csv(data_path + booth_key_filename)
turnstiles1

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/30/2019,03:00:00,REGULAR,7290785,2470635
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/30/2019,07:00:00,REGULAR,7290795,2470652
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/30/2019,11:00:00,REGULAR,7290860,2470733
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/30/2019,15:00:00,REGULAR,7291051,2470808
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/30/2019,19:00:00,REGULAR,7291396,2470862
...,...,...,...,...,...,...,...,...,...,...,...
205919,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/06/2019,04:00:00,REGULAR,5554,420
205920,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/06/2019,08:00:00,REGULAR,5554,420
205921,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/06/2019,12:00:00,REGULAR,5554,420
205922,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/06/2019,16:00:00,REGULAR,5554,420


In [5]:
turnstiles1.columns = [x.strip() for x in turnstiles1.columns]
turnstiles1.groupby(['DESC']).DESC.count()

DESC
RECOVR AUD       864
REGULAR       205060
Name: DESC, dtype: int64

In [6]:
# First drop a few columns that we don't use
turnstiles1.drop(['DIVISION'], axis = 1, inplace=True)
turnstiles1.drop(['SCP'], axis = 1, inplace=True)

In [7]:
# Initialize the variables we're using for cleaning and summarizing
delta_entry = [0]
delta_exit = [0]
date_list = [dt.datetime.strptime(turnstiles1['DATE'][0], '%m/%d/%Y').date()]
day_list = [date_list[0].strftime('%A')]

# Loop through the rows of the dataframe
for row in range(1, len(turnstiles1)) :
    # Assume we're going to have a 0 change for each row
    entry_update = 0
    exit_update = 0
    
    # If we're still dealing with the same station
    if((turnstiles1['STATION'][row] == turnstiles1['STATION'][row - 1]) &
       (turnstiles1['DESC'][row] == 'REGULAR') &
       (turnstiles1['DATE'][row] == turnstiles1['DATE'][row - 1])) :
        
        # And we havn't changed a date
        if(turnstiles1['ENTRIES'][row] > turnstiles1['ENTRIES'][row - 1]) :
            entry_update = (turnstiles1['ENTRIES'][row] - turnstiles1['ENTRIES'][row - 1])
        
        if(turnstiles1['EXITS'][row] > turnstiles1['EXITS'][row - 1]) :
            exit_update = (turnstiles1['EXITS'][row] - turnstiles1['EXITS'][row - 1])
        
        # Check for 'outliers'
        if(entry_update > 20000) :
            entry_update = 0
            
        if(exit_update > 20000) :
            exit_update = 0
    
    # Update the lists
    delta_entry.append(entry_update)
    delta_exit.append(exit_update)
    date_list.append(dt.datetime.strptime(turnstiles1['DATE'][row], '%m/%d/%Y').date())
    day_list.append(date_list[row].strftime('%A'))

# Add new columns to the data frame for the newly calculated data
turnstiles1['deltaEntry'] = delta_entry
turnstiles1['deltaExit'] = delta_exit
turnstiles1['datetime'] = date_list
turnstiles1['weekday'] = day_list;

In [8]:
high_traffic_in = pd.DataFrame(turnstiles1.groupby(["STATION"]).deltaEntry.sum().sort_values(ascending=False).head(25))
high_traffic_in = set(high_traffic_in.index)

In [9]:
HighTrafficOut = pd.DataFrame(turnstiles1.groupby(['STATION']).deltaExit.sum().sort_values(ascending=False).head(25))
high_traffic_out = set(HighTrafficOut.index)

In [10]:
high_traffic_stations = high_traffic_out.union(high_traffic_in)


In [11]:
high_traffic_df = turnstiles1[turnstiles1['STATION'].isin(high_traffic_stations)]
high_traffic_df.groupby(['STATION', 'weekday']).deltaEntry.sum().sort_values(ascending=False).head(100)

STATION          weekday  
34 ST-PENN STA   Thursday     176926
                 Wednesday    175600
                 Tuesday      173211
                 Friday       167729
                 Monday       162510
                               ...  
50 ST            Friday        54898
PATH NEW WTC     Friday        54326
50 ST            Wednesday     53754
FLUSHING-MAIN    Monday        53361
42 ST-PORT AUTH  Saturday      52784
Name: deltaEntry, Length: 100, dtype: int64

In [14]:
turnstiles1

,C/A,UNIT,STATION,LINENAME,DATE,TIME,DESC,ENTRIES,EXITS,deltaEntry,deltaExit,datetime,weekday
0,A002,R051,59 ST,NQR456W,10/26/2019,00:00:00,REGULAR,7247322,2455491,0,0,2019-10-26,Saturday
1,A002,R051,59 ST,NQR456W,10/26/2019,04:00:00,REGULAR,7247336,2455499,14,8,2019-10-26,Saturday
2,A002,R051,59 ST,NQR456W,10/26/2019,08:00:00,REGULAR,7247351,2455532,15,33,2019-10-26,Saturday
3,A002,R051,59 ST,NQR456W,10/26/2019,12:00:00,REGULAR,7247463,2455623,112,91,2019-10-26,Saturday
4,A002,R051,59 ST,NQR456W,10/26/2019,16:00:00,REGULAR,7247755,2455679,292,56,2019-10-26,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206041,TRAM2,R469,RIT-ROOSEVELT,R,11/01/2019,05:00:00,REGULAR,5554,420,0,0,2019-11-01,Friday
206042,TRAM2,R469,RIT-ROOSEVELT,R,11/01/2019,09:00:00,REGULAR,5554,420,0,0,2019-11-01,Friday
206043,TRAM2,R469,RIT-ROOSEVELT,R,11/01/2019,13:00:00,REGULAR,5554,420,0,0,2019-11-01,Friday
206044,TRAM2,R469,RIT-ROOSEVELT,R,11/01/2019,17:00:00,REGULAR,5554,420,0,0,2019-11-01,Friday
